In [21]:
%load_ext fortranmagic
%matplotlib inline

import numpy as np

The fortranmagic extension is already loaded. To reload it, use:
  %reload_ext fortranmagic


In [5]:
%%fortran

subroutine centerpos(pos, dim, n_atom)
    implicit none
    
    integer, intent(in) :: dim, n_atom
    real(8), intent(inout), dimension(0:n_atom-1, 0:dim-1) :: pos
    real(8), dimension(0:dim-1) :: pos_avg
    integer :: i, j
    
    pos_avg = sum(pos, 1) / dble(n_atom)      !sums over the first axis
    
    do i = 0, n_atom-1
        do j = 0, dim-1
            pos(i,j) = pos(i,j) - pos_avg(j)
        end do
    end do

end subroutine centerpos

In [21]:
a = np.arange(15, dtype = 'float').reshape(5,3) 
a.dtype

dtype('float64')

In [15]:
dim = 3
n_atom = 5
type(dim), type(n_atom)

(int, int)

case in python

In [42]:
a - a.mean(axis = 0)

array([[-6., -6., -6.],
       [-3., -3., -3.],
       [ 0.,  0.,  0.],
       [ 3.,  3.,  3.],
       [ 6.,  6.,  6.]])

we have to change a from row major to column major first

In [39]:
a_fort = np.asfortranarray(a)

In [41]:
centerpos(a_fort, dim, n_atom)
a_fort

array([[-6., -6., -6.],
       [-3., -3., -3.],
       [ 0.,  0.,  0.],
       [ 3.,  3.,  3.],
       [ 6.,  6.,  6.]])

now we have output array of order='F', to convert it back to order='C' that we can apply numpy-base operation

In [31]:
a_fort.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [35]:
a_c = np.ascontiguousarray(a_fort)

In [36]:
a_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

like python, fortran allows array operations.

In [43]:
%%fortran

subroutine centerpos_v(pos, dim, n_atom)
    implicit none
    
    integer, intent(in) :: dim, n_atom
    real(8), intent(inout), dimension(0:n_atom-1, 0:dim-1) :: pos
    real(8), dimension(0:dim-1) :: pos_avg
    integer :: i
    
    pos_avg = sum(pos, 1) / dble(n_atom)      !sums over the first axis
    
    do i = 0, n_atom-1
        pos(i,:) = pos(i,:) - pos_avg(:)
    end do

end subroutine centerpos_v

In [45]:
a_fort = np.asfortranarray(a)
print(a_fort)
centerpos_v(a_fort, dim, n_atom)
a_fort

[[  0.   1.   2.]
 [  3.   4.   5.]
 [  6.   7.   8.]
 [  9.  10.  11.]
 [ 12.  13.  14.]]


array([[-6., -6., -6.],
       [-3., -3., -3.],
       [ 0.,  0.,  0.],
       [ 3.,  3.,  3.],
       [ 6.,  6.,  6.]])

## now lets see which one is the fastest

In [52]:
%%timeit
a - np.mean(a, axis = 0)

10000 loops, best of 3: 31.6 µs per loop


In [47]:
%%timeit
a_fort = np.asfortranarray(a)
centerpos(a_fort, dim, n_atom)
a_c = np.ascontiguousarray(a_fort)
a_c

100000 loops, best of 3: 12.9 µs per loop


despite the time wasted in converting array order

In [48]:
a_fort = np.asfortranarray(a)
%timeit centerpos(a_fort, dim, n_atom)
np.ascontiguousarray(a_fort)

The slowest run took 10.90 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 603 ns per loop


array([[-6., -6., -6.],
       [-3., -3., -3.],
       [ 0.,  0.,  0.],
       [ 3.,  3.,  3.],
       [ 6.,  6.,  6.]])

now how about fortran's array operation.

In [50]:
a_fort = np.asfortranarray(a)
%timeit centerpos_v(a_fort, dim, n_atom)
np.ascontiguousarray(a_fort)

The slowest run took 10.29 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 602 ns per loop


array([[-6., -6., -6.],
       [-3., -3., -3.],
       [ 0.,  0.,  0.],
       [ 3.,  3.,  3.],
       [ 6.,  6.,  6.]])

## fortran is almost 50 times faster than numpy (pure computation)

lets modify centerpos_v a little bit

In [5]:
a = np.arange(15, dtype = 'float').reshape(3,5) 
dim = 3
n_atom = 5
a

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.]])

In [16]:
%%fortran

subroutine centerpos_v1(pos, dim, n_atom)
    implicit none
    
    integer, intent(in) :: dim, n_atom
    real(8), intent(inout), dimension(0:dim-1, 0:n_atom-1) :: pos
    real(8), dimension(0:dim-1) :: pos_avg
    integer :: i
    
    pos_avg = sum(pos, 2) / dble(n_atom)      !sums over the second axis
    
    do i = 0, n_atom-1
        pos(:,i) = pos(:,i) - pos_avg(:)
    end do

end subroutine centerpos_v1

In [15]:
%%timeit
a - a.mean(axis = 1).reshape(3,1)*np.ones((1,5), dtype = 'float')

10000 loops, best of 3: 49.9 µs per loop


In [20]:
a_fort = np.asfortranarray(a)
%timeit centerpos_v1(a_fort, dim, n_atom)
a_fort

The slowest run took 10.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 576 ns per loop


array([[-2., -1.,  0.,  1.,  2.],
       [-2., -1.,  0.,  1.,  2.],
       [-2., -1.,  0.,  1.,  2.]])

conclusion: if we rerange the array that the main calculation in fortran is done in column, it will be a little bit faster!